In [7]:
import os
import random
import cv2 as cv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def bgr2rgb(img):
    return img[...,::-1]

def load_color_img(path):
    return np.array( bgr2rgb(cv.imread(path)), dtype=np.float32 )

def load_color_img2(path):
    img = cv.imread(path)
    return img.transpose(2, 1, 0)

data_path = os.getcwd() + "/eye_data" + "/set1"
eyes_path = data_path + "/images" + "/"
vessels_path = data_path + "/vessels" + "/"

yes_path = data_path + "/classes/yes_original" + "/"
no_path = data_path + "/classes/no_original" + "/"

yeses = os.listdir(yes_path)
noes = os.listdir(no_path)

random.shuffle(yeses)
random.shuffle(noes)

yeses = yeses[:100000]
noes = noes[:100000]

test = load_color_img(yes_path + yeses[0])
height = test.shape[0]
width = test.shape[1]
channels = test.shape[2]

labels = {
    'yes' : 1,
    'no' : 0
}

train_x = []
train_y = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())

N = len(yeses) + len(noes)
train_x = np.empty((N, height, width, channels), dtype=np.float32)
train_y = np.empty(N, dtype=np.uint8)

idx = 0
print("loading yeses...")
for file in yeses:
    
    im = load_color_img(yes_path+file)
    #im_s = sess.run(tf.image.per_image_standardization(im))
    
    train_x[idx] = im
    train_y[idx] = 1
    idx += 1

    if idx % 2500 == 0:
        print(idx)
print('yes loaded.')

print("loading noes...")
for file in noes:
    
    im = load_color_img(no_path+file)
    #im_s = sess.run(tf.image.per_image_standardization(im))
    
    train_x[idx] = im
    train_y[idx] = 0
    idx += 1

    if idx % 2500 == 0:
        print(idx)
print('no loaded.')

train_x /= 255.0

#test_x /= 255.0

loading yeses...
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
yes loaded.
loading noes...
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
200000
no loaded.


In [8]:
from keras.utils import to_categorical

train_y_cat = to_categorical(train_y)
train_y_cat.shape

(200000, 2)

In [9]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_label, valid_label = train_test_split(train_x, train_y_cat, test_size=0.25, random_state=13)
train_x.shape, valid_x.shape

((150000, 33, 33, 3), (50000, 33, 33, 3))

In [10]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint

batch_size = 64
epochs = 150
num_classes = len(labels)
input_shape = train_x[0].shape

fashion_model = Sequential()
fashion_model.add(Conv2D(48, kernel_size=(3, 3),activation='linear',padding='same',input_shape=input_shape))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(BatchNormalization())
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
#fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(BatchNormalization())
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='relu'))
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

filepath = data_path + "/models_okidoki/" + "weights.{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                             verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

fashion_model.summary()
fashion_train = fashion_model.fit(train_x, train_label, 
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  verbose=1,
                                  callbacks=callbacks_list,
                                  validation_data=(valid_x, valid_label))
fashion_model.save("okidoki model.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 33, 33, 48)        1344      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 33, 33, 48)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 33, 33, 48)        192       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 48)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 17, 17, 48)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 17, 17, 64)        27712     
_________________________________________________________________
batch_normalization_6 (Batch (None, 17, 17, 64)        256       
__________

150000/150000 [==============================] - 2722s 18ms/step - loss: 0.2011 - acc: 0.9247 - val_loss: 0.2261 - val_acc: 0.9145

Epoch 00027: val_acc did not improve
Epoch 28/150
150000/150000 [==============================] - 2742s 18ms/step - loss: 0.1995 - acc: 0.9252 - val_loss: 0.1952 - val_acc: 0.9264

Epoch 00028: val_acc improved from 0.92518 to 0.92642, saving model to C:\Users\mariu\Documents\Eye Blood Vessels/eye_data/set1/models_okidoki/weights.28-0.93.hdf5
Epoch 29/150
150000/150000 [==============================] - 2752s 18ms/step - loss: 0.1988 - acc: 0.9259 - val_loss: 0.3419 - val_acc: 0.8440

Epoch 00029: val_acc did not improve
Epoch 30/150
150000/150000 [==============================] - 2831s 19ms/step - loss: 0.1975 - acc: 0.9261 - val_loss: 0.1987 - val_acc: 0.9271

Epoch 00030: val_acc improved from 0.92642 to 0.92710, saving model to C:\Users\mariu\Documents\Eye Blood Vessels/eye_data/set1/models_okidoki/weights.30-0.93.hdf5
Epoch 31/150
150000/150000 [===

KeyboardInterrupt: 